In [2]:
import pandas as pd
import polars as pl
import numpy as np
import gc
from matplotlib import pyplot as plt
import matplotlib.cm as cm
from sklearn.model_selection import StratifiedGroupKFold

# Config

In [3]:
class CONFIG:
    target_col = "responder_6"
    lag_cols_original = ["date_id", "symbol_id"] + [f"responder_{idx}" for idx in range(9)]
    lag_cols_rename = { f"responder_{idx}" : f"responder_{idx}_lag_1" for idx in range(9)}
    valid_ratio = 0.09
    start_dt = 1450
    path = f"/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet"

In [4]:
#1 698 max date 
# 120 for validation => 1 578 left => 200 for train => 1 378

# Load training data

In [5]:
# Use last 2 parquets
train = pl.scan_parquet(
    CONFIG.path
    # f"/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet"
).select(
    pl.int_range(pl.len(), dtype=pl.UInt32).alias("id"),
    pl.all(),
).with_columns(
    (pl.col(CONFIG.target_col)*2).cast(pl.Int32).alias("label"),
).filter(
    pl.col("date_id").gt(CONFIG.start_dt)
)

# Create Lags data from training data

In [6]:
lags = train.select(pl.col(CONFIG.lag_cols_original))
lags = lags.rename(CONFIG.lag_cols_rename)
lags = lags.with_columns(
    date_id = pl.col('date_id') + 1,  # lagged by 1 day
    )
lags = lags.group_by(["date_id", "symbol_id"], maintain_order=True).last()  # pick up last record of previous date
lags

# Merge training data and lags data

In [7]:
train = train.join(lags, on=["date_id", "symbol_id"],  how="left")

# Split training data and validation data

In [10]:
len_train   = train.select(pl.col("date_id")).collect().shape[0]
valid_records = int(len_train * CONFIG.valid_ratio)
len_ofl_mdl = len_train - valid_records
last_tr_dt  = train.select(pl.col("date_id")).collect().row(len_ofl_mdl)[0]

print(f"\n len_train = {len_train}")
print(f"\n len_ofl_mdl = {len_ofl_mdl}")
print(f"\n---> Last offline train date = {last_tr_dt}\n")

training_data = train.filter(pl.col("date_id").le(last_tr_dt))
validation_data   = train.filter(pl.col("date_id").gt(last_tr_dt))


 len_train = 9144696

 len_ofl_mdl = 8321674

---> Last offline train date = 1677



In [9]:
# validation_data

# Save data as parquets

In [11]:
training_data.collect().\
write_parquet(
    f"training.parquet", partition_by = "date_id",
)

In [12]:
validation_data.collect().\
write_parquet(
    "validation.parquet", partition_by = "date_id",
)